## register_buffer

参考：

1. [What is the difference between 'register_buffer' and 'register_parameter' of 'nn.Module'](https://discuss.pytorch.org/t/what-is-the-difference-between-register-buffer-and-register-parameter-of-nn-module/32723)

2. [Pytorch模型中的parameter与buffer](https://zhuanlan.zhihu.com/p/89442276)

在Pytorch中，一种模型保存方法如下：

In [1]:
import torch

In [3]:
class MyModel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return self.linear(x)

In [6]:
model = MyModel(2, 3)
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[-0.4406,  0.0370],
        [ 0.1972,  0.5418],
        [ 0.5899, -0.0691]])), ('linear.bias', tensor([ 0.2596, -0.5622, -0.2093]))])


可以看到model.state.dict的返回对象是一个OrderedDict，它以键值对的形式包含了模型中需要保存下来的参数。Mymodel中就是weight, bias

接下来看parameter 和 buffer

模型中需要保存下来的参数有两种：

1. 经过反向传播，需要被optimizer更新的，称为parameter

2. 不经过反向传播 称为buffer 比如 transformer中的postion embedding

第一种参数我们可以通过 model.parameters() 返回；

第二种参数我们可以通过 model.buffers() 返回。

In [7]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        buffer = torch.randn(2, 3)
        self.register_buffer('my_buffer', buffer)
        self.param = torch.nn.Parameter(torch.rand(2, 3))

    def forward(self, x):
        pass
        

可以看到模型保存的参数，既包括parameters, 也包括buffers

In [8]:
model = MyModel()
print(model.state_dict())

OrderedDict([('param', tensor([[0.9412, 0.0178, 0.0433],
        [0.8280, 0.6318, 0.2142]])), ('my_buffer', tensor([[ 1.5445,  2.1983, -1.7092],
        [-1.8460,  0.5065, -0.5747]]))])


In [9]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[0.9412, 0.0178, 0.0433],
        [0.8280, 0.6318, 0.2142]], requires_grad=True)


In [10]:
for param in model.buffers():
    print(param)

tensor([[ 1.5445,  2.1983, -1.7092],
        [-1.8460,  0.5065, -0.5747]])


In [11]:
model.cuda()
print(model.state_dict())

OrderedDict([('param', tensor([[0.9412, 0.0178, 0.0433],
        [0.8280, 0.6318, 0.2142]], device='cuda:0')), ('my_buffer', tensor([[ 1.5445,  2.1983, -1.7092],
        [-1.8460,  0.5065, -0.5747]], device='cuda:0'))])


总结：

1. 模型中需要进行更新的参数注册为 parameter, 不进行更新的设置为buffer.

2. 模型保存的参数是 model.state_dict() 返回的 OrderedDict

3. 模型进行设备移动时，模型中注册的参数 parameter buffer 会同时移动